# Classification
### Step 1: resize/crop images
Requires having run: `beescrape.py`
Depends on yolov5 and `shutil`

In [29]:
import os
from shutil import copyfile
import shutil

# Directories
DATA_DIR = '/home/wilber/Documents/RESEARCH/research/beespotter'
OUT_DIR = '/m2docs/res/data'
CROPPED_PATH = '/m2docs/res/cropped_imgs'

%rm -R /m2docs/res/data/*

# Args:
VAL_SIZE = 100
TEST_SIZE = 100
MAX_IMGS = 1000

# Species with >= 1000 images:
labels = ['Apis_mellifera','Bombus_impatiens','Bombus_auricomus','Bombus_bimaculatus','Bombus_griseocollis']

errnum = 0
for label in labels:  # for each type of bee
    classID = labels.index(label)
    
    input_path = os.path.join(DATA_DIR, label)
    
    # Attempt to create directories:
    if not os.path.exists(OUT_DIR + '/train/' + label):
        os.makedirs(OUT_DIR + '/train/' + label)
    if not os.path.exists(OUT_DIR + '/test/' + label):
        os.makedirs(OUT_DIR + '/test/' + label)
    if not os.path.exists(OUT_DIR + '/validation/' + label):
        os.makedirs(OUT_DIR + '/validation/' + label)
        
    train_path = os.path.join(OUT_DIR,'train/'+label+'/')
    test_path  = os.path.join(OUT_DIR + 'test/'+label+'/')
    valid_path = os.path.join(OUT_DIR,'validation/'+label+'/')
    
    index = 0
    im_list = os.listdir(input_path)
    for img in im_list:
        try:
            if index < VAL_SIZE:
                copyfile(os.path.join(CROPPED_PATH,img), os.path.join(valid_path,img))
            elif index < (VAL_SIZE + TEST_SIZE):
                copyfile(os.path.join(CROPPED_PATH,img), os.path.join(test_path,img)) 
            elif index < (VAL_SIZE + TEST_SIZE + MAX_IMGS):
                copyfile(os.path.join(CROPPED_PATH,img), os.path.join(train_path,img))
            else:
                break
        except:
            errnum += 1
            index -= 1
            pass
        index += 1
    print(label, "errors", errnum)
    

print("Images moved. Errors suppresed: ", errnum)


Apis_mellifera errors 2285
Bombus_impatiens errors 4523
Bombus_auricomus errors 5274
Bombus_bimaculatus errors 6377
Bombus_griseocollis errors 8308
Images moved. Errors suppresed:  8308
